# RAG 평가 개요
- RAG 평가란 RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정이다. 
- **평가 요소**
    - **검색 단계 평가**
        - 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.
    - **생성 단계 평가**
        - 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.
- **평가 방법**
    - 온/오프라인 평가
        1. **오프라인 평가**
            - 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정한다.
        2. **온라인 평가**
            - 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가한다.
    - 정량적/정성적 평가
        1. 정량적 평가
            - 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가한다.
        2. 정성적 평가
            - 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가한다.

# [RAGAS](https://www.ragas.io/)
- RAGAS는 RAG 파이프라인을 **정량적 으로 평가하는** 오픈소스 프레임 워크이다. 
- RAGAS 문서: https://docs.ragas.io/en/stable/
## 설치
- `pip install ragas rapidfuzz`

## RAGAS 평가 지표 개요
![ragas_score](figures/ragas_score.png)
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.
- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- 이러한 지표들은 RAG 파이프라인의 성능을 다각도로 평가하는 데 활용된다.
![RAGAS_score2](figures/RAGAS_score2.png)

# GUIDE
- 위 그림에서 box안에 파란색 빨간색이 평가 대상으로 보면된다.
- 즉 generation은 Answer에 대한 평가지표이므로 => answer와 context: faithfulness, answer와 user input이 answer relevancy 이런식으로 설명한다.

## Precision@K 
- K 상위 K개 라고 하는데 이건 top-k 의 상위 k를 말한다.
- precision@3 이면 상위 3개중 찾은 문서가 몇개 포함되있나
  - 정답: [1, 2, 3, 4] , 찾은 문서: [2, 1, 5, 6] - precision@3 은 상위 3개중 2와 1 두개가 포함되 있으므로 2/3 이된다.

> - claim 뜻: 주장, 요청

## 주요 평가지표
### Generation 평가
- LLM이 생성한 답변에 대한 평가
  
#### Faithfulness (신뢰성)
- 생성된 답변이 얼마나 주어진 검색 문서들(context)를 잘 반영해서 생성되었는지 평가한다. 할루시네이션에 대한 평가라고 할 수 있다. 
- 점수범위: **0 ~ 1** (1에 가까울수록 좋음)
- 답변에 포함된 모든 주장이 context에서 얼마나 추출 가능한지를 확인한다.

##### 평가 방법
1. Answer에서 주장 구문(claim statement)들을 생성(추출)한다. (주장이란, 질문(user input)과 관련된 내용)
    - 예) 
        - **질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요? 
        - **LLM 답변**: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 3000만명이다.
2. 각 주장들을 context로 부터 추론 가능한지 판단한다. 이를 바탕으로 faithfulness 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다. .... 한국의 인구는 5000만명이고 서울에 1000만이 살고 있다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 추론가능한 주장.
            - 한국의 인구는 3000만명이다. -> context에서 추론 불가능한 주장.
3. **Faithfulness score** 를 계산한다. 총 주장 수 중에서 context로 부터 추론가능한 주장의 개수.    
    - 예)
        - Faithfulness Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 유추할 수있다.)
    - LLM 답변에서 주장을 추출 하는 것과 각 주장이 context에서 추론 가능한 지를 판단하는 것은 LLM 을 활용한다.
- 공식
    $$
    \text{Faithfulness Score}\;=\;\cfrac{\text{주어진\;context\;에서\;추론할\;수\;있는\;주장의\;개수}}{\text{총\;주장\;개수}}
    $$

### Answer relevancy (답변 적합성)
- 생성된 답변이 질문(user input)에 얼마나 잘 부합하는 지를 평가한다.
- 점수 범위: -1~1 (1에 가까울수록 좋음)
- LLM이 생성한 답변을 기반으로 질문들을 생성한다. 이렇게 생성한 질문들과 실제 질문(user input) 간의 유사도를 측정한다.

#### 평가방법
1. LLM이 생성한 답변을 기반으로 질문들을 생성한다.
    - 예) 
        - **LLM** 답변: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
2. 실제 질문과 생성한 질문간의 코사인 유사도를 측정한다. 그 평균이 최종 점수가 된다.
    - 예)
        - **실제 질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요?
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
- 공식
  $$
    \cfrac{1}{N} \sum_{i=1}^{N} \text{cosine\_similarity}(q_{\text{user}_{_i}}, q_{\text{generated}})
  $$

## Retrieval 평가
Vector store에서 검색한 context에 대한 평가

### Context Precision
- 검색된 문서(context)들 중 질문과 관련 있는 것들이 얼마나 상위 순위**에 있는 지 평가.
- 점수 범위: 0~1 (1에 가까울수록 좋음)


#### 평가방법

- 공식
$$
 \text{Context\;Precision@K} = \frac{\sum_{k=1}^{K} \left( \text{Precision@k} \times v_k \right)}{\ 상위\;K개\;결과에서의\;관련\;항목\;수}
$$
$$
 \text{Precision@k} = \frac{\text{True\;positive@k}}{(\text{True\;positive@k} + \text{False\;positive@k})} \\
$$
- $\text{Precision@k}$: 개별 문서에 대한 Precision
- K: context 의 개수(chuck 수)
- $v_k$: 관련성 여부로 0 또는 1. (0: 관련 없음, 1: 관련 있음)

#### 예시
- 질문과 context 관련성 예
    - 질문: 한국의 수도는 어디이고 인구는 얼마나 되나요?
    - 높은 정밀도 context
        - 한국의 수도는 서울이고 인구는 5000명 입니다. 
        - 한국의 수도는 서울입니다.
        - 한국은 동아시아에 위치해 있는 국가로 수도는 서울입니다.
        - 한국의 인구는 5000만명 입니다.
    - 낮은 정밀도 context
        - 한국은 동아시아에 위치한 국가입니다.
        - 한국의 K-pop은 전 세계적으로 유명합니다.
        - 비빔밥, 불고기는 한국의 대표적인 음식입니다.
    - **높은 정밀도의 context이 상위 순위에 위치했으면 높은 점수를 받는다.**
- 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정
```bash
Precision@1 = 1/1 = 1.0    # True positive@1/(True positive@1 + False positive@1).  1/1(1번 문서 계산 시에는 1개 문서만 있으므로 분모가 1이 된다.)
Precision@2 = 1/2 = 0.5     
Precision@3 = 2/3 ≈ 0.67    
Precision@4 = 3/4 = 0.75
Precision@5 = 3/5 = 0.6
```
- vk의 값
    - 1번째: $v_1 = 1$
    - 2번째: $v_2 = 0$
    - 3번째: $v_3 = 1$
    - 4번째: $v_4 = 1$
    - 5번째: $v_5 = 0$

- Context Precision@5
$$
\text{Context\;Precision@5} = \frac{(1.0 \times 1) + (0.5 \times 0) + (0.67 \times 1) + (0.75 \times 1) + (0.6 \times 0)}{3} = \frac{1.0 + 0 + 0.67 + 0.75 + 0}{3} ≈ 0.807
$$

# GUIDE
- 예를 들어, 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정하면:
```bash
Precision@1 = 1/1 = 1.0     # 1등 -> 1/1  --> 1 등이 관련 없는 것이면 전체 평균 점수가 확 낮아 질 수있다.
Precision@2 = 1/2 = 0.5     # 2등 -> 1/2  (1, 2등 두개 중에 하나만 관련(1번)이 있다.)
Precision@3 = 2/3 ≈ 0.67    # 3등 -> 2/3  (1, 2, 3 등 세개 중에 두개만 관련(1,3번)이 있다.)
Precision@4 = 3/4 = 0.75
Precision@5 = 3/5 = 0.
```
왜 분모가 1, 2, 3, 4, 5 -> 그 순위에는 상위순위와 자신밖에 없으므로 그 개수가 분모가됨. (3등은 1, 2, 3 등 세개 문서로.)

### Context Recall (문맥 재현률)
- 검색된 문서(context)가 얼마나 정답(ground-truth)의 정보를 포함있는 지 평가하는 지표
- 점수 범위: 0~1 (1에 가까울수록 좋음)
- **정답(ground truth)의 각 주장(claim)이 검색된 context와 얼마나 일치**하는지 계산함.

#### 평가방법
1. 정답에서 주장(claim)들을 생성(추출)한다.
    - 예) 
        - **정답**: 한국의 수도는 서울이고 인구수는 5000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 5000만명이다.
2. 각 주장(claim)의 정보를 검색된 contexts에서 찾을 수 있는지 판별한다. 이를 바탕으로 context recall 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 찾을 수 있다.
            - 한국의 인구는 5000만명이다. -> context에서 찾을 수 없다.
3. **Context Recall Score** 를 계산한다. 총 주장 수 중에서 context로 부터 찾을 수 있는 주장의 개수.
    - 예)
        - Context Recall Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 찾을 수 있다.)

- 공식
    $$
    \text{Context Recall Score}\;=\;\cfrac{\text{GT의\;주장\;중\;주어진\;context\;에서\;찾을\;수\;있는\;주장의\;개수}}{\text{GT의\;총\;주장\;개수}}
    $$ 

# RAGAS 평가 실습

In [ ]:
# !uv pip install ragas rapidfuzz
# 설치 후 커널 재시작

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, SparseVectorParams, VectorParams
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()


In [ ]:
# ##############################################################
# 데이터 준비
##############################################################

def load_and_split_olympic_data(file_path="data/olympic_wiki.md"):
    with open(file_path, "r", encoding="utf-8") as fr:
        olympic_text = fr.read()

    # Split
    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
        ],
        # strip_headers=False, # 문서에 header 포함 여부(default: True - 제거)
    )

    return splitter.split_text(olympic_text)

In [ ]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

def get_vectorstore(collection_name: str = "olympic_info_wiki"):


    dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

    client = QdrantClient(host="localhost", port=6333)

    # 컬렉션 삭제
    if client.collection_exists(collection_name):
        result = client.delete_collection(collection_name=collection_name)

    # 컬렉션 생성
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=3072, distance=Distance.COSINE),
      
    )

    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,    
        embedding=dense_embeddings
    )
    
    ######################################
    # Document들 추가
    ######################################
    documents = load_and_split_olympic_data()
    vectorstore.add_documents(documents=documents)

    return vectorstore


def get_retriever(vectorstore, k: int = 5):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k}
    )
    return retriever

In [ ]:
vectorstore = get_vectorstore()

retriever = get_retriever(vectorstore)
retriever

In [ ]:
################################################################################
# 평가할 RAG Chain
################################################################################

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document


vectorstore = get_vectorstore()
retriever = get_retriever(vectorstore)

prompt_txt = """# Instruction:
당신은 정보제공을 목적으로하는 유능한 AI Assistant 입니다.
주어진 context의 내용을 기반으로 질문에 답변을 합니다.
Context에 질문에 대한 명확한 정보가 있는 경우 그것을 바탕으로 답변을 합니다.
Context에 질문에 대한 명확한 정보가 없는 경우 "정보가 부족해 답을 할 수없습니다." 라고 답합니다.
절대 추측이나 일반 상식을 바탕으로 답을 하거나 Context 없는 내용을 만들어서 답변해서는 안됩니다.

# Context:
{context}

# 질문:
{query}
"""
prompt = ChatPromptTemplate.from_template(
    template=prompt_txt
)


def format_docs(documents:list)->str:
    """
    VectorStore에 조회한 문서들에서 내용(page_content)만 추출해서 str으로 합쳐서 반환.
    VectorStore의 검색결과인 List[Document]를 받아서 Document들에서 page_content의 내용만 추출한다.
    
    Args:
        documents(list[Document]): [Document(..), Document(...), ..]}
    Returns:
        str: 각 문서의 내용을 "\n\n"으로 연결한 string
    """
    return "\n\n".join(doc.page_content for doc in documents)

model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

chain = {
    "context":retriever | format_docs,
    "query":RunnablePassthrough()
} | prompt | model | parser


In [ ]:
chain.invoke("1회 올림픽은 언제 어디서 열렸지")

# RAGAS 를 이용해 평가를 위한 합성 데이터 셋 만들기

- 평가 데이터셋 구성
  - `user_input`: 사용자 질문
  - `retrieved_contexts`: Vectorstore에서 검색한 context
  - `response`: LLM의 응답
  - `reference`: 정답

## TestsetGenerator
- **문서(retrieved_contexts)를 기준**으로 **질문**, **정답** 을 생성한다.
- 평가할 LLM으로 생성된 질문을 넣어 답변을 추출하여 데이터셋을 구성한다.

In [ ]:
# 주피터노트북 환경에서 비동기적 처리 위해

import nest_asyncio
nest_asyncio.apply()